In [84]:
!pip install pypdf unstructured markdown faiss-cpu > /dev/null

In [85]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader

# 定义文件路径
pdf_file = "./data/2023级计算机科学与技术学术硕士研究生培养方案.pdf"
md_file1 = "./data/历史沿革.md"
md_file2 = "./data/学院简介.md"

# 加载文件内容
pdf_loader = PyPDFLoader(file_path=pdf_file)
md_loader1 = UnstructuredMarkdownLoader(file_path=md_file1)
md_loader2 = UnstructuredMarkdownLoader(file_path=md_file2)

In [86]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
def split(chunk_size, doc):
    text_splitter = RecursiveCharacterTextSplitter(
    	chunk_size = chunk_size,
    	chunk_overlap  = chunk_size // 10,
	)
    return text_splitter.split_documents(doc)

In [87]:
pdf_documents = split(chunk_size=250, doc=pdf_loader.load())
md_documents1 = split(chunk_size=50, doc=md_loader1.load())
md_documents2 = split(chunk_size=100, doc=md_loader2.load())
documents = pdf_documents + md_documents1 + md_documents2
print(f'chunk nums = {len(documents)}')

chunk nums = 86


In [88]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import DashScopeEmbeddings
import os

embeddings = DashScopeEmbeddings(
    model="text-embedding-v1", dashscope_api_key=os.environ["DASHSCOPE_API_KEY"]
)


In [89]:
db = FAISS.from_documents(documents, embeddings) # 构建数据库
retriever = db.as_retriever( # 转换为检索器 返回最相关的k个文档
    search_kwargs = {
        'k': 2
	}
)

In [90]:
result = retriever.get_relevant_documents("学院共有几次晋级ICPC国际大学生程序设计竞赛全球总决赛，分别是哪几年？")
result

[Document(metadata={'source': './data/学院简介.md'}, page_content='际大学生程序设计竞赛亚洲赛区和中国大学生程序设计竞赛的金/银/铜奖，并于2007年、2010年、2018年、2019年和2020年共五次成功晋级ICPC国际大学生程序设计竞赛全球总决赛，曾获得全国“挑'),
 Document(metadata={'source': './data/学院简介.md'}, page_content='决赛，曾获得全国“挑战杯”一等奖和三等奖、中国国际大学生创新大赛全国金奖、全国大学生物联网设计竞赛全国一等奖、“创青春”全国大学生创业大赛金奖等，在NTIRE国际去雾挑战赛、SemEval国际语义评测')]

In [91]:
from langchain_community.llms import Tongyi
llm = Tongyi( model_name="qwen1.5-1.8b-chat", temperature=0.7, top_p=0.7 )
print( llm.invoke("你好吗？") )

我是一个大型语言模型，没有情感和个人感觉。但我可以回答各种问题和提供帮助。有什么我可以帮助你的吗？


In [124]:
import langchain
langchain.debug = False
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever,verbose=True)
qa.combine_documents_chain.llm_chain.prompt.template = '''
Use the following pieces of context to answer the question at the end without any additional information. if you don't know the answer, just say that you don't know, don't try to make up an answer. 

{context}

Question: {question}
Answer:
'''


In [109]:
# query = '学院共有几次晋级ICPC国际大学生程序设计竞赛全球总决赛，分别是哪几年？'
# qa.invoke(query)

In [125]:
from langchain.evaluation.qa import QAEvalChain
eval_chain = QAEvalChain.from_llm(llm)


examples = [
    {
        'query':'学院在未来的发展目标是什么？',
        'answer': '学院致力于建设成为世界一流的计算机科学与技术学院'
	},
    {
        'query':'2019年7月6日，华东师范大学成立了哪个学院？',
        'answer': '计算机科学与技术学院'
	},
    {
        'query':'硕士研究生的总学分要求是多少？',
        'answer': '硕士研究生的总学分要求是23学分'
	}
]

predictions = qa.batch(inputs=examples)



> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...


KeyError: 'request'

In [ ]:
graded_outputs = eval_chain.evaluate(examples, predictions)

[chain/start] [chain:QAEvalChain] Entering Chain run with input:
{
  "input_list": [
    {
      "query": "学院在未来的发展目标是什么？",
      "answer": "学院致力于建设成为世界一流的计算机科学与技术学院",
      "result": " 学院在未来的发展目标是把我院建设成为世界一流学院。为了实现这个目标，学院将遵循“育人、文明、发展”责任使命，聚焦“教育+”、“生态+”、“健康+”、“智能+”、“国际+”五大行动计划，通过不断创新和实践，提高人才培养质量，提升教育教学水平，推进学科建设，加强国际合作交流，促进全面发展，为社会培养具有创新能力、批判性思维、人文素养和全球视野的应用型人才。同时，学院还将坚持走内涵式发展道路，注重学科交叉融合，推动科技成果产业化，以科技创新驱动学院持续高质量发展，引领社会进步和科技进步，为实现中华民族伟大复兴中国梦作出更大贡献。"
    }
  ]
}
[llm/start] [chain:QAEvalChain > llm:Tongyi] Entering LLM run with input:
{
  "prompts": [
    "You are a teacher grading a quiz.\nYou are given a question, the student's answer, and the true answer, and are asked to score the student answer as either CORRECT or INCORRECT.\n\nExample Format:\nQUESTION: question here\nSTUDENT ANSWER: student's answer here\nTRUE ANSWER: true answer here\nGRADE: CORRECT or INCORRECT here\n\nGrade the student answers based ONLY on their factual accuracy. Ignore differences in punctuation and phrasing 

In [ ]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]["query"])
    print("Real Answer: " + predictions[i]["answer"])
    print("Predicted Answer: " + predictions[i]["result"])
    print("Predicted Grade: " + graded_outputs[i]["results"])
    print()


Example 0:
Question: 学院在未来的发展目标是什么？
Real Answer: 学院致力于建设成为世界一流的计算机科学与技术学院
Predicted Answer:  学院在未来的发展目标是把我院建设成为世界一流学院。为了实现这个目标，学院将遵循“育人、文明、发展”责任使命，聚焦“教育+”、“生态+”、“健康+”、“智能+”、“国际+”五大行动计划，通过不断创新和实践，提高人才培养质量，提升教育教学水平，推进学科建设，加强国际合作交流，促进全面发展，为社会培养具有创新能力、批判性思维、人文素养和全球视野的应用型人才。同时，学院还将坚持走内涵式发展道路，注重学科交叉融合，推动科技成果产业化，以科技创新驱动学院持续高质量发展，引领社会进步和科技进步，为实现中华民族伟大复兴中国梦作出更大贡献。
Predicted Grade: CORRECT

